In [18]:
import json
from datasets import load_dataset
import pandas as pd

# 데이터셋 경로와 n-best 예측값 파일 경로 설정
dataset_path = "/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/data/default/validation/data-00000-of-00001.arrow"
nbest_pred_path = "/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/outputs/original_default_bz=16_lr=1.6764783497920226e-05_eval_nbest_predictions.json"

# 데이터셋 로드
dataset = load_dataset('arrow', data_files=dataset_path, split='train')

# n-best 예측값 로드
with open(nbest_pred_path, 'r', encoding='utf-8') as f:
    nbest_preds = json.load(f)

# 틀린 예측을 저장할 리스트 초기화
wrong_cases = []

# 데이터셋의 각 예제에 대해 반복
for example in dataset:
    example_id = example['id']
    # 정답은 리스트 형태로 저장되어 있다고 가정
    correct_answers = example['answers']['text']
    
    # 해당 예제의 n-best 예측값 가져오기
    preds = nbest_preds.get(example_id, [])
    if not preds:
        continue  # 예측값이 없는 경우 건너뜀
    
    # 최상위 예측값
    top_pred = preds[0]['text']
    
    # 최상위 예측값이 정답과 일치하는지 확인 (대소문자 구분 없이 비교)
    if top_pred.strip().lower() not in [ans.strip().lower() for ans in correct_answers]:
        # 틀린 경우 n-best 예측값 저장
        nbest_details = []
        for idx, pred in enumerate(preds, 1):
            detail = (
                f"**Prediction {idx}:**<br>"
                f"- Text: {pred['text']}<br>"
                f"- Probability: {pred['probability']:.4f}<br>"
                f"- Start Logit: {pred['start_logit']:.4f}<br>"
                f"- End Logit: {pred['end_logit']:.4f}"
            )
            nbest_details.append(detail)
        
        # 모든 예측값을 <br><br>로 구분하여 하나의 문자열로 합침
        nbest_combined = "<br><br>".join(nbest_details)
        
        wrong_cases.append({
            'ID': example_id,
            '정답': "; ".join(correct_answers),
            'N-best 예측값': nbest_combined
        })

# 틀린 사례가 없는 경우 메시지 출력
if not wrong_cases:
    print("모델이 틀린 예측을 하지 않았습니다.")
else:
    # 마크다운 표 헤더 생성
    headers = ["ID", "정답", "N-best 예측값"]
    header_line = "| " + " | ".join(headers) + " |"
    separator_line = "| " + " | ".join(["---"] * len(headers)) + " |"
    
    # 각 행 생성
    rows = []
    for case in wrong_cases:
        row = f"| {case['ID']} | {case['정답']} | {case['N-best 예측값']} |"
        rows.append(row)
    
    # 전체 표 생성
    markdown_table = "\n".join([header_line, separator_line] + rows)
    print(markdown_table)


| ID | 정답 | N-best 예측값 |
| --- | --- | --- |
| mrc-0-005106 | 옥음방송 | **Prediction 1:**<br>- Text: 쇼와 천황의 옥음방송<br>- Probability: 0.9660<br>- Start Logit: 8.1817<br>- End Logit: 7.8816<br><br>**Prediction 2:**<br>- Text: 옥음방송<br>- Probability: 0.0328<br>- Start Logit: 4.7979<br>- End Logit: 7.8816<br><br>**Prediction 3:**<br>- Text: 쇼와 천황의 옥음방송(라디오 방송)<br>- Probability: 0.0011<br>- Start Logit: 8.1817<br>- End Logit: 1.1315<br><br>**Prediction 4:**<br>- Text: 천황의 옥음방송<br>- Probability: 0.0000<br>- Start Logit: -1.8940<br>- End Logit: 7.8816<br><br>**Prediction 5:**<br>- Text: 옥음방송(라디오 방송)<br>- Probability: 0.0000<br>- Start Logit: 4.7979<br>- End Logit: 1.1315 |
| mrc-1-000414 | 복잡한 감염병 | **Prediction 1:**<br>- Text: 열병의 감염원을 밝히기 위한<br>- Probability: 0.8422<br>- Start Logit: 9.2939<br>- End Logit: 4.1028<br><br>**Prediction 2:**<br>- Text: 복잡한 감염병<br>- Probability: 0.0978<br>- Start Logit: 6.4807<br>- End Logit: 4.7630<br><br>**Prediction 3:**<br>- Text: 열병의 감염원<br>- Probability: 0.0280<

| ID | 정답 | N-best 예측값 |
| --- | --- | --- |
| mrc-0-005106 | 옥음방송 | **Prediction 1:**<br>- Text: 쇼와 천황의 옥음방송<br>- Probability: 0.9660<br>- Start Logit: 8.1817<br>- End Logit: 7.8816<br><br>**Prediction 2:**<br>- Text: 옥음방송<br>- Probability: 0.0328<br>- Start Logit: 4.7979<br>- End Logit: 7.8816<br><br>**Prediction 3:**<br>- Text: 쇼와 천황의 옥음방송(라디오 방송)<br>- Probability: 0.0011<br>- Start Logit: 8.1817<br>- End Logit: 1.1315<br><br>**Prediction 4:**<br>- Text: 천황의 옥음방송<br>- Probability: 0.0000<br>- Start Logit: -1.8940<br>- End Logit: 7.8816<br><br>**Prediction 5:**<br>- Text: 옥음방송(라디오 방송)<br>- Probability: 0.0000<br>- Start Logit: 4.7979<br>- End Logit: 1.1315 |
| mrc-1-000414 | 복잡한 감염병 | **Prediction 1:**<br>- Text: 열병의 감염원을 밝히기 위한<br>- Probability: 0.8422<br>- Start Logit: 9.2939<br>- End Logit: 4.1028<br><br>**Prediction 2:**<br>- Text: 복잡한 감염병<br>- Probability: 0.0978<br>- Start Logit: 6.4807<br>- End Logit: 4.7630<br><br>**Prediction 3:**<br>- Text: 열병의 감염원<br>- Probability: 0.0280<br>- Start Logit: 9.2939<br>- End Logit: 0.7003<br><br>**Prediction 4:**<br>- Text: 열병의 감염원을 밝히<br>- Probability: 0.0212<br>- Start Logit: 9.2939<br>- End Logit: 0.4221<br><br>**Prediction 5:**<br>- Text: 감염병<br>- Probability: 0.0107<br>- Start Logit: 4.2681<br>- End Logit: 4.7630 |
| mrc-0-002875 | 스페인 | **Prediction 1:**<br>- Text: 프랑스<br>- Probability: 0.2856<br>- Start Logit: 8.9723<br>- End Logit: 10.2397<br><br>**Prediction 2:**<br>- Text: 프랑스<br>- Probability: 0.2143<br>- Start Logit: 8.6146<br>- End Logit: 10.3101<br><br>**Prediction 3:**<br>- Text: 스페인 펠리페 4세의 딸이었던 마리아 테레사가 프랑스<br>- Probability: 0.2134<br>- Start Logit: 8.6810<br>- End Logit: 10.2397<br><br>**Prediction 4:**<br>- Text: 스페인<br>- Probability: 0.1515<br>- Start Logit: 8.6810<br>- End Logit: 9.8968<br><br>**Prediction 5:**<br>- Text: 스페인 펠리페 4세의 딸이었던 마리아 테레사가 프랑스<br>- Probability: 0.1352<br>- Start Logit: 8.1543<br>- End Logit: 10.3101 |
| mrc-1-000724 | 1965년 | **Prediction 1:**<br>- Text: 1963년<br>- Probability: 0.9785<br>- Start Logit: 10.3136<br>- End Logit: 8.8349<br><br>**Prediction 2:**<br>- Text: 1963<br>- Probability: 0.0129<br>- Start Logit: 10.3136<br>- End Logit: 4.5055<br><br>**Prediction 3:**<br>- Text: 1965년<br>- Probability: 0.0071<br>- Start Logit: 7.4116<br>- End Logit: 6.8065<br><br>**Prediction 4:**<br>- Text: 1965<br>- Probability: 0.0013<br>- Start Logit: 7.4116<br>- End Logit: 5.1508<br><br>**Prediction 5:**<br>- Text: 1965년 캐나다 연방 선거는 캐나다에서 27번째로 치러진 연방 선거로, 캐나다 하원의원 265석을 선출하기 위해 1965년<br>- Probability: 0.0002<br>- Start Logit: 7.4116<br>- End Logit: 3.2484 |
| mrc-0-002138 | 아리크 부케 | **Prediction 1:**<br>- Text: 쿠빌라이<br>- Probability: 0.7385<br>- Start Logit: 10.8543<br>- End Logit: 10.3569<br><br>**Prediction 2:**<br>- Text: 둘째동생 쿠빌라이<br>- Probability: 0.2611<br>- Start Logit: 9.8146<br>- End Logit: 10.3569<br><br>**Prediction 3:**<br>- Text: 아리크 부케<br>- Probability: 0.0004<br>- Start Logit: 7.2386<br>- End Logit: 6.4024<br><br>**Prediction 4:**<br>- Text: 둘째동생<br>- Probability: 0.0000<br>- Start Logit: 9.8146<br>- End Logit: 0.9180<br><br>**Prediction 5:**<br>- Text: 아리크 부케<br>- Probability: 0.0000<br>- Start Logit: 4.5012<br>- End Logit: 4.7210 |
| mrc-0-003115 | 미타케성 | **Prediction 1:**<br>- Text: 이이노야 성<br>- Probability: 0.9999<br>- Start Logit: 12.2046<br>- End Logit: 11.5321<br><br>**Prediction 2:**<br>- Text: 이이노야<br>- Probability: 0.0000<br>- Start Logit: 12.2046<br>- End Logit: 1.6014<br><br>**Prediction 3:**<br>- Text: 노야 성<br>- Probability: 0.0000<br>- Start Logit: 1.4209<br>- End Logit: 11.5321<br><br>**Prediction 4:**<br>- Text: 이이노야 성이 평소, 이이 씨가 전쟁중이 아닐 때 생활하던 거성이고, 미타케성이<br>- Probability: 0.0000<br>- Start Logit: 12.2046<br>- End Logit: 0.3569<br><br>**Prediction 5:**<br>- Text: 성<br>- Probability: 0.0000<br>- Start Logit: 0.7151<br>- End Logit: 11.5321 |
| mrc-0-000785 | 유럽 포르투갈어 | **Prediction 1:**<br>- Text: 브라질 포르투갈어<br>- Probability: 0.9686<br>- Start Logit: 8.4626<br>- End Logit: 8.6636<br><br>**Prediction 2:**<br>- Text: 브라질 포르투갈어<br>- Probability: 0.0301<br>- Start Logit: 6.1353<br>- End Logit: 7.5210<br><br>**Prediction 3:**<br>- Text: 브라질<br>- Probability: 0.0009<br>- Start Logit: 8.4626<br>- End Logit: 1.7219<br><br>**Prediction 4:**<br>- Text: 브라질 포르투갈어와 유럽 포르투갈어<br>- Probability: 0.0003<br>- Start Logit: 8.4626<br>- End Logit: 0.5081<br><br>**Prediction 5:**<br>- Text: 유럽 포르투갈어<br>- Probability: 0.0000<br>- Start Logit: 1.2683<br>- End Logit: 0.5081 |
| mrc-0-001519 | 메이저 릿지 | **Prediction 1:**<br>- Text: 릿지당<br>- Probability: 0.7998<br>- Start Logit: 10.8272<br>- End Logit: 11.2967<br><br>**Prediction 2:**<br>- Text: 릿지당<br>- Probability: 0.2001<br>- Start Logit: 9.9603<br>- End Logit: 10.7782<br><br>**Prediction 3:**<br>- Text: 당<br>- Probability: 0.0000<br>- Start Logit: 0.5015<br>- End Logit: 11.2967<br><br>**Prediction 4:**<br>- Text: 릿지<br>- Probability: 0.0000<br>- Start Logit: 10.8272<br>- End Logit: 0.7431<br><br>**Prediction 5:**<br>- Text: 릿지당을 규탄하면서 그 당원 한 사람을 죽였다. 릿지당<br>- Probability: 0.0000<br>- Start Logit: 9.9603<br>- End Logit: 1.5066 |
| mrc-0-002457 | 요희 | **Prediction 1:**<br>- Text: 요희의 오라버니<br>- Probability: 0.2759<br>- Start Logit: -0.6483<br>- End Logit: -1.3872<br><br>**Prediction 2:**<br>- Text: 조경숙왕은 강도역왕의 첩이면서 의붓아들인 강도왕 유건<br>- Probability: 0.2384<br>- Start Logit: -1.2022<br>- End Logit: -0.9795<br><br>**Prediction 3:**<br>- Text: 요희(淖姬)<br>- Probability: 0.1950<br>- Start Logit: -1.0108<br>- End Logit: -1.3717<br><br>**Prediction 4:**<br>- Text: 조경숙왕은 강도역왕의 첩이면서 의붓아들인 강도왕 유건과 간통한 요희(淖姬)<br>- Probability: 0.1611<br>- Start Logit: -1.2022<br>- End Logit: -1.3717<br><br>**Prediction 5:**<br>- Text: 요희<br>- Probability: 0.1296<br>- Start Logit: -0.6483<br>- End Logit: -2.1433 |
| mrc-1-001023 | 도고쿠 | **Prediction 1:**<br>- Text: 사이넨지<br>- Probability: 0.5404<br>- Start Logit: 11.3895<br>- End Logit: 10.0591<br><br>**Prediction 2:**<br>- Text: 사이넨지(西念寺)<br>- Probability: 0.4573<br>- Start Logit: 11.3895<br>- End Logit: 9.8922<br><br>**Prediction 3:**<br>- Text: 도고쿠에 지은 암자 가운데 하나였던 사이넨지<br>- Probability: 0.0011<br>- Start Logit: 5.2103<br>- End Logit: 10.0591<br><br>**Prediction 4:**<br>- Text: 도고쿠에 지은 암자 가운데 하나였던 사이넨지(西念寺)<br>- Probability: 0.0009<br>- Start Logit: 5.2103<br>- End Logit: 9.8922<br><br>**Prediction 5:**<br>- Text: 신란이 도고쿠에 지은 암자 가운데 하나였던 사이넨지<br>- Probability: 0.0003<br>- Start Logit: 3.8494<br>- End Logit: 10.0591 |
| mrc-0-001022 | 몽키 D. 가프 | **Prediction 1:**<br>- Text: 루피의 할아버지인 몽키 D. 가프<br>- Probability: 0.6530<br>- Start Logit: 12.3047<br>- End Logit: 11.5204<br><br>**Prediction 2:**<br>- Text: 몽키 D. 가프<br>- Probability: 0.2405<br>- Start Logit: 11.3057<br>- End Logit: 11.5204<br><br>**Prediction 3:**<br>- Text: 루피의 할아버지<br>- Probability: 0.1063<br>- Start Logit: 12.3047<br>- End Logit: 9.7050<br><br>**Prediction 4:**<br>- Text: 할아버지인 몽키 D. 가프<br>- Probability: 0.0002<br>- Start Logit: 3.9365<br>- End Logit: 11.5204<br><br>**Prediction 5:**<br>- Text: D. 가프<br>- Probability: 0.0000<br>- Start Logit: 2.1545<br>- End Logit: 11.5204 |
| mrc-0-001150 | 조류 | **Prediction 1:**<br>- Text: 해양 생태계의 착생식물<br>- Probability: 0.7988<br>- Start Logit: 4.7344<br>- End Logit: 5.9805<br><br>**Prediction 2:**<br>- Text: 해양 생태계의 착생식물은 흔히 해초나 조류의 표면에서 자라나는 조류<br>- Probability: 0.1198<br>- Start Logit: 4.7344<br>- End Logit: 4.0831<br><br>**Prediction 3:**<br>- Text: 조류<br>- Probability: 0.0666<br>- Start Logit: 4.1480<br>- End Logit: 4.0831<br><br>**Prediction 4:**<br>- Text: 해양 착생식물의 생태는 지상 착생식물의 생태와는 다르다. 해양 생태계의 착생식물<br>- Probability: 0.0098<br>- Start Logit: 0.3334<br>- End Logit: 5.9805<br><br>**Prediction 5:**<br>- Text: 해양<br>- Probability: 0.0050<br>- Start Logit: 4.7344<br>- End Logit: 0.9027 |
| mrc-0-001161 | 작가 베게티우스 | **Prediction 1:**<br>- Text: 베게티우스<br>- Probability: 0.7553<br>- Start Logit: 8.0565<br>- End Logit: 9.0396<br><br>**Prediction 2:**<br>- Text: 로마 작가 베게티우스<br>- Probability: 0.2387<br>- Start Logit: 6.9044<br>- End Logit: 9.0396<br><br>**Prediction 3:**<br>- Text: 베게티우스<br>- Probability: 0.0026<br>- Start Logit: 5.3588<br>- End Logit: 6.0829<br><br>**Prediction 4:**<br>- Text: 후기 로마 작가 베게티우스<br>- Probability: 0.0023<br>- Start Logit: 2.2573<br>- End Logit: 9.0396<br><br>**Prediction 5:**<br>- Text: 로마 작가 베게티우스<br>- Probability: 0.0011<br>- Start Logit: 4.5063<br>- End Logit: 6.0829 |
| mrc-0-002468 | 쇠망치 | **Prediction 1:**<br>- Text: 망치<br>- Probability: 0.9527<br>- Start Logit: 8.7903<br>- End Logit: 9.1511<br><br>**Prediction 2:**<br>- Text: 쇠망치를 챙겨 피해자의 객실 문을 열고 객실로 들어가 망치<br>- Probability: 0.0422<br>- Start Logit: 5.6730<br>- End Logit: 9.1511<br><br>**Prediction 3:**<br>- Text: 쇠망치<br>- Probability: 0.0051<br>- Start Logit: 5.6730<br>- End Logit: 7.0298<br><br>**Prediction 4:**<br>- Text: 치를 챙겨 피해자의 객실 문을 열고 객실로 들어가 망치<br>- Probability: 0.0000<br>- Start Logit: -2.7414<br>- End Logit: 9.1511<br><br>**Prediction 5:**<br>- Text: 망치로 피해자를 살해<br>- Probability: 0.0000<br>- Start Logit: 8.7903<br>- End Logit: -2.8881 |
| mrc-1-000995 | 홋카이도 | **Prediction 1:**<br>- Text: 홋카이도로<br>- Probability: 0.5322<br>- Start Logit: 7.4636<br>- End Logit: 8.2963<br><br>**Prediction 2:**<br>- Text: 시모키타 반도의 가키자키로 이동하였고, 이후에 홋카이도로<br>- Probability: 0.3802<br>- Start Logit: 7.1274<br>- End Logit: 8.2963<br><br>**Prediction 3:**<br>- Text: 시모키타 반도의 가키자키<br>- Probability: 0.0583<br>- Start Logit: 7.1274<br>- End Logit: 6.4206<br><br>**Prediction 4:**<br>- Text: 시모키타 반도<br>- Probability: 0.0223<br>- Start Logit: 7.1274<br>- End Logit: 5.4584<br><br>**Prediction 5:**<br>- Text: 가키자키로 이동하였고, 이후에 홋카이도로<br>- Probability: 0.0071<br>- Start Logit: 3.1413<br>- End Logit: 8.2963 |
| mrc-1-001814 | 독일인민당 | **Prediction 1:**<br>- Text: 독일 인민당<br>- Probability: 0.9992<br>- Start Logit: 10.4649<br>- End Logit: 10.2928<br><br>**Prediction 2:**<br>- Text: 독일인민당<br>- Probability: 0.0005<br>- Start Logit: 6.0314<br>- End Logit: 7.2036<br><br>**Prediction 3:**<br>- Text: 독일 민주당이 그를 당의 고위직에 받아들이지 않자 스스로 독일 인민당<br>- Probability: 0.0001<br>- Start Logit: 1.4710<br>- End Logit: 10.2928<br><br>**Prediction 4:**<br>- Text: 스스로 독일 인민당<br>- Probability: 0.0001<br>- Start Logit: 1.1629<br>- End Logit: 10.2928<br><br>**Prediction 5:**<br>- Text: 독일인민당<br>- Probability: 0.0000<br>- Start Logit: 4.6674<br>- End Logit: 5.4758 |
| mrc-0-004342 | 가루아 | **Prediction 1:**<br>- Text: 이슬비(가루아)<br>- Probability: 0.4127<br>- Start Logit: -1.9143<br>- End Logit: -1.4839<br><br>**Prediction 2:**<br>- Text: 잦은 이슬비(가루아)<br>- Probability: 0.2229<br>- Start Logit: -2.5304<br>- End Logit: -1.4839<br><br>**Prediction 3:**<br>- Text: 이슬비<br>- Probability: 0.1932<br>- Start Logit: -1.9143<br>- End Logit: -2.2430<br><br>**Prediction 4:**<br>- Text: 잦은 이슬비<br>- Probability: 0.1043<br>- Start Logit: -2.5304<br>- End Logit: -2.2430<br><br>**Prediction 5:**<br>- Text: 가랑비<br>- Probability: 0.0670<br>- Start Logit: -2.5592<br>- End Logit: -2.6575 |
| mrc-0-002931 | 나말여초 | **Prediction 1:**<br>- Text: 나말여초 불상<br>- Probability: 0.9997<br>- Start Logit: 8.3577<br>- End Logit: 7.8572<br><br>**Prediction 2:**<br>- Text: 나말여초<br>- Probability: 0.0002<br>- Start Logit: 8.3577<br>- End Logit: -0.4386<br><br>**Prediction 3:**<br>- Text: 불상<br>- Probability: 0.0001<br>- Start Logit: -1.4274<br>- End Logit: 7.8572<br><br>**Prediction 4:**<br>- Text: 말여초 불상<br>- Probability: 0.0000<br>- Start Logit: -2.3984<br>- End Logit: 7.8572<br><br>**Prediction 5:**<br>- Text: 나<br>- Probability: 0.0000<br>- Start Logit: 8.3577<br>- End Logit: -3.2001 |
| mrc-1-000449 | 여정현(呂正鉉) | **Prediction 1:**<br>- Text: 여정현<br>- Probability: 0.6579<br>- Start Logit: 11.8205<br>- End Logit: 8.8197<br><br>**Prediction 2:**<br>- Text: 여정현(呂正鉉)<br>- Probability: 0.3386<br>- Start Logit: 11.8205<br>- End Logit: 8.1553<br><br>**Prediction 3:**<br>- Text: 여정현(呂正鉉)과 경주 이씨(慶州李氏)<br>- Probability: 0.0025<br>- Start Logit: 11.8205<br>- End Logit: 3.2550<br><br>**Prediction 4:**<br>- Text: 여정현(呂正鉉)과 경주 이씨<br>- Probability: 0.0010<br>- Start Logit: 11.8205<br>- End Logit: 2.3221<br><br>**Prediction 5:**<br>- Text: 경주 이씨(慶州李氏)<br>- Probability: 0.0000<br>- Start Logit: 2.3372<br>- End Logit: 3.2550 |
| mrc-0-003677 | 퀘이커 교 | **Prediction 1:**<br>- Text: 퀘이커<br>- Probability: 0.9546<br>- Start Logit: 11.1976<br>- End Logit: 10.6490<br><br>**Prediction 2:**<br>- Text: 퀘이커 교도<br>- Probability: 0.0453<br>- Start Logit: 11.1976<br>- End Logit: 7.6018<br><br>**Prediction 3:**<br>- Text: 커<br>- Probability: 0.0000<br>- Start Logit: 0.5961<br>- End Logit: 10.6490<br><br>**Prediction 4:**<br>- Text: 커 교도<br>- Probability: 0.0000<br>- Start Logit: 0.5961<br>- End Logit: 7.6018<br><br>**Prediction 5:**<br>- Text: 청교도<br>- Probability: 0.0000<br>- Start Logit: 3.4570<br>- End Logit: 2.8693 |
| mrc-0-005412 | 지분 | **Prediction 1:**<br>- Text: 지분의 일부<br>- Probability: 0.4113<br>- Start Logit: 8.8786<br>- End Logit: 7.3842<br><br>**Prediction 2:**<br>- Text: 지분 10%를 인수하고, YG엔터테인먼트와 양현석 대표가 엠넷미디어 지분의 일부<br>- Probability: 0.3042<br>- Start Logit: 8.5770<br>- End Logit: 7.3842<br><br>**Prediction 3:**<br>- Text: 지분 10%<br>- Probability: 0.1658<br>- Start Logit: 8.5770<br>- End Logit: 6.7770<br><br>**Prediction 4:**<br>- Text: 엠넷미디어 지분의 일부<br>- Probability: 0.1150<br>- Start Logit: 7.6039<br>- End Logit: 7.3842<br><br>**Prediction 5:**<br>- Text: 지분<br>- Probability: 0.0038<br>- Start Logit: 8.8786<br>- End Logit: 2.6962 |
| mrc-0-000561 | 독일 | **Prediction 1:**<br>- Text: 독일군의 대참패<br>- Probability: 0.7994<br>- Start Logit: 8.9470<br>- End Logit: 10.9127<br><br>**Prediction 2:**<br>- Text: 스탈린그라드 전투에서 독일군의 대참패<br>- Probability: 0.1913<br>- Start Logit: 7.5167<br>- End Logit: 10.9127<br><br>**Prediction 3:**<br>- Text: 독일의 궁극적인 몰락<br>- Probability: 0.0063<br>- Start Logit: 8.2484<br>- End Logit: 6.7624<br><br>**Prediction 4:**<br>- Text: 대참패<br>- Probability: 0.0026<br>- Start Logit: 3.2356<br>- End Logit: 10.9127<br><br>**Prediction 5:**<br>- Text: 코카서스 공격안은 결국 스탈린그라드 전투에서 독일군의 대참패<br>- Probability: 0.0004<br>- Start Logit: 1.2982<br>- End Logit: 10.9127 |
| mrc-0-001704 | 가리타 히사노리 | **Prediction 1:**<br>- Text: 가리타 히사노리에<br>- Probability: 0.8541<br>- Start Logit: 11.8410<br>- End Logit: 9.2882<br><br>**Prediction 2:**<br>- Text: 가리타 히사노리에<br>- Probability: 0.1430<br>- Start Logit: 11.2287<br>- End Logit: 8.1136<br><br>**Prediction 3:**<br>- Text: 가리타 히사노<br>- Probability: 0.0022<br>- Start Logit: 11.8410<br>- End Logit: 3.3366<br><br>**Prediction 4:**<br>- Text: 가리타 히사<br>- Probability: 0.0005<br>- Start Logit: 11.8410<br>- End Logit: 1.9118<br><br>**Prediction 5:**<br>- Text: 가리타 히사노<br>- Probability: 0.0002<br>- Start Logit: 11.2287<br>- End Logit: 1.3075 |
| mrc-0-003118 | 그 자체가 계급 모순을 가지는 체제 | **Prediction 1:**<br>- Text: 사회주의<br>- Probability: 0.7395<br>- Start Logit: 2.3904<br>- End Logit: 2.9727<br><br>**Prediction 2:**<br>- Text: 계급 모순을 가지는 체제<br>- Probability: 0.1788<br>- Start Logit: 2.0729<br>- End Logit: 1.8704<br><br>**Prediction 3:**<br>- Text: 사회주의로 나아갈 수밖에 없다는 것이 종속이론의 주장이라는 것이다. 그러나 쿠에바는 평등하고 조화로운 자본주의<br>- Probability: 0.0446<br>- Start Logit: 2.3904<br>- End Logit: 0.1647<br><br>**Prediction 4:**<br>- Text: 자본주의는 그 자체가 계급 모순을 가지는 체제<br>- Probability: 0.0248<br>- Start Logit: 0.0988<br>- End Logit: 1.8704<br><br>**Prediction 5:**<br>- Text: 그 자체가 계급 모순을 가지는 체제<br>- Probability: 0.0122<br>- Start Logit: -0.6078<br>- End Logit: 1.8704 |
| mrc-0-000905 | 경순왕 | **Prediction 1:**<br>- Text: 경애왕<br>- Probability: 0.6954<br>- Start Logit: 7.2971<br>- End Logit: 7.4465<br><br>**Prediction 2:**<br>- Text: 경애왕과 경순왕<br>- Probability: 0.2325<br>- Start Logit: 7.2971<br>- End Logit: 6.3507<br><br>**Prediction 3:**<br>- Text: 경순왕<br>- Probability: 0.0378<br>- Start Logit: 5.4797<br>- End Logit: 6.3507<br><br>**Prediction 4:**<br>- Text: 태조가 동경을 방문할 때면 경애왕<br>- Probability: 0.0173<br>- Start Logit: 3.6056<br>- End Logit: 7.4465<br><br>**Prediction 5:**<br>- Text: 경애왕과 경순왕은 그를 극진히 대접하고 회유하였다. 경애왕<br>- Probability: 0.0170<br>- Start Logit: 7.2971<br>- End Logit: 3.7356 |
| mrc-0-003947 | 바르톨로메오 브뤼기에르 | **Prediction 1:**<br>- Text: 파리 외방전교회의 바르톨로메오 브뤼기에르 신부<br>- Probability: 0.6225<br>- Start Logit: 10.8469<br>- End Logit: 9.9489<br><br>**Prediction 2:**<br>- Text: 바르톨로메오 브뤼기에르 신부<br>- Probability: 0.3695<br>- Start Logit: 10.3253<br>- End Logit: 9.9489<br><br>**Prediction 3:**<br>- Text: 파리 외방전교회의 바르톨로메오 브뤼기에르<br>- Probability: 0.0043<br>- Start Logit: 10.8469<br>- End Logit: 4.9829<br><br>**Prediction 4:**<br>- Text: 바르톨로메오 브뤼기에르<br>- Probability: 0.0026<br>- Start Logit: 10.3253<br>- End Logit: 4.9829<br><br>**Prediction 5:**<br>- Text: 브뤼기에르 신부<br>- Probability: 0.0011<br>- Start Logit: 4.5231<br>- End Logit: 9.9489 |
| mrc-1-001522 | 손권 | **Prediction 1:**<br>- Text: 대제 손권<br>- Probability: 0.7020<br>- Start Logit: 10.9049<br>- End Logit: 11.6368<br><br>**Prediction 2:**<br>- Text: 손권<br>- Probability: 0.2976<br>- Start Logit: 10.0466<br>- End Logit: 11.6368<br><br>**Prediction 3:**<br>- Text: 대제 손권에게 아뢨으며, 손권<br>- Probability: 0.0002<br>- Start Logit: 10.9049<br>- End Logit: 3.6810<br><br>**Prediction 4:**<br>- Text: 대제 손<br>- Probability: 0.0001<br>- Start Logit: 10.9049<br>- End Logit: 2.8941<br><br>**Prediction 5:**<br>- Text: 손권에게 아뢨으며, 손권<br>- Probability: 0.0001<br>- Start Logit: 10.0466<br>- End Logit: 3.6810 |
| mrc-0-004307 | 350년 | **Prediction 1:**<br>- Text: 콘스탄티누스 1세<br>- Probability: 0.7902<br>- Start Logit: 3.8831<br>- End Logit: 2.5893<br><br>**Prediction 2:**<br>- Text: 콘스탄티누스 1세와 황후인 파우스타<br>- Probability: 0.1255<br>- Start Logit: 3.8831<br>- End Logit: 0.7491<br><br>**Prediction 3:**<br>- Text: 마그넨티우스<br>- Probability: 0.0464<br>- Start Logit: 2.0053<br>- End Logit: 1.6317<br><br>**Prediction 4:**<br>- Text: 콘스탄티우스 2세<br>- Probability: 0.0349<br>- Start Logit: 2.1630<br>- End Logit: 1.1908<br><br>**Prediction 5:**<br>- Text: 파우스타<br>- Probability: 0.0030<br>- Start Logit: 0.1355<br>- End Logit: 0.7491 |
| mrc-0-002471 | 펜실베이니아주 출신 | **Prediction 1:**<br>- Text: 제임스 뷰캐넌<br>- Probability: 0.9954<br>- Start Logit: 8.4999<br>- End Logit: 8.3856<br><br>**Prediction 2:**<br>- Text: 펜실베이니아주<br>- Probability: 0.0044<br>- Start Logit: 5.8009<br>- End Logit: 5.6553<br><br>**Prediction 3:**<br>- Text: 뷰캐넌<br>- Probability: 0.0002<br>- Start Logit: 0.0361<br>- End Logit: 8.3856<br><br>**Prediction 4:**<br>- Text: 펜실베이니아<br>- Probability: 0.0000<br>- Start Logit: 5.8009<br>- End Logit: -0.4191<br><br>**Prediction 5:**<br>- Text: 사일러스 라이트<br>- Probability: 0.0000<br>- Start Logit: 1.0622<br>- End Logit: 1.7327 |
| mrc-0-004197 | 깃털 셔틀콕의 타구감을 선호하고, 또한 플라스틱보다 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문이다 | **Prediction 1:**<br>- Text: 깃털 셔틀콕의 타구감을 선호하고, 또한 플라스틱보다 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문<br>- Probability: 0.9103<br>- Start Logit: 6.4036<br>- End Logit: 5.1567<br><br>**Prediction 2:**<br>- Text: 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문<br>- Probability: 0.0496<br>- Start Logit: 3.4946<br>- End Logit: 5.1567<br><br>**Prediction 3:**<br>- Text: 깃털 셔틀콕의 타구감을 선호하고, 또한 플라스틱보다 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문<br>- Probability: 0.0243<br>- Start Logit: 4.3347<br>- End Logit: 3.6029<br><br>**Prediction 4:**<br>- Text: 깃털 셔틀콕의 타구감을 선호<br>- Probability: 0.0144<br>- Start Logit: 6.4036<br>- End Logit: 1.0075<br><br>**Prediction 5:**<br>- Text: 깃털 셔틀콕의 타구감을 선호<br>- Probability: 0.0014<br>- Start Logit: 4.3347<br>- End Logit: 0.7529 |
| mrc-1-000839 | 목판 | **Prediction 1:**<br>- Text: 등<br>- Probability: 0.9883<br>- Start Logit: 5.8091<br>- End Logit: 5.4325<br><br>**Prediction 2:**<br>- Text: 코뿔소의 등<br>- Probability: 0.0089<br>- Start Logit: 1.1017<br>- End Logit: 5.4325<br><br>**Prediction 3:**<br>- Text: 등에 뒤틀린 작은 뿔<br>- Probability: 0.0016<br>- Start Logit: 5.8091<br>- End Logit: -1.0244<br><br>**Prediction 4:**<br>- Text: 딱딱한 판으로 몸이 둘러싸인 동물로 묘사하였다. 또, 코뿔소의 등<br>- Probability: 0.0012<br>- Start Logit: -0.8958<br>- End Logit: 5.4325<br><br>**Prediction 5:**<br>- Text: 코뿔소의 등에 뒤틀린 작은 뿔<br>- Probability: 0.0000<br>- Start Logit: 1.1017<br>- End Logit: -1.0244 |
| mrc-0-003564 | 혼묘지 | **Prediction 1:**<br>- Text: 혼묘지(本妙寺)<br>- Probability: 0.5767<br>- Start Logit: 11.3106<br>- End Logit: 11.5807<br><br>**Prediction 2:**<br>- Text: 에도의 큰 사찰인 혼묘지(本妙寺)<br>- Probability: 0.3056<br>- Start Logit: 10.6757<br>- End Logit: 11.5807<br><br>**Prediction 3:**<br>- Text: 혼묘지<br>- Probability: 0.0751<br>- Start Logit: 11.3106<br>- End Logit: 9.5428<br><br>**Prediction 4:**<br>- Text: 에도의 큰 사찰인 혼묘지<br>- Probability: 0.0398<br>- Start Logit: 10.6757<br>- End Logit: 9.5428<br><br>**Prediction 5:**<br>- Text: 혼묘지<br>- Probability: 0.0027<br>- Start Logit: 8.4825<br>- End Logit: 9.0505 |
| mrc-1-000291 | 혹 | **Prediction 1:**<br>- Text: 이질<br>- Probability: 1.0000<br>- Start Logit: 9.2885<br>- End Logit: 9.3530<br><br>**Prediction 2:**<br>- Text: 이질이 발생<br>- Probability: 0.0000<br>- Start Logit: 9.2885<br>- End Logit: -1.6750<br><br>**Prediction 3:**<br>- Text: 스텐코프의 혹<br>- Probability: 0.0000<br>- Start Logit: 2.6493<br>- End Logit: 1.0447<br><br>**Prediction 4:**<br>- Text: 의무관이었던 스텐코프의 혹<br>- Probability: 0.0000<br>- Start Logit: 1.6829<br>- End Logit: 1.0447<br><br>**Prediction 5:**<br>- Text: 스텐코프의 혹을 치료<br>- Probability: 0.0000<br>- Start Logit: 2.6493<br>- End Logit: -1.0705 |
| mrc-0-003033 | 귀인 평향과 행동의 상관 관계 | **Prediction 1:**<br>- Text: 특정한 귀인 평향과 행동의 상관 관계<br>- Probability: 0.9512<br>- Start Logit: 8.3458<br>- End Logit: 6.9484<br><br>**Prediction 2:**<br>- Text: 귀인 평향과 행동의 상관 관계<br>- Probability: 0.0324<br>- Start Logit: 4.9661<br>- End Logit: 6.9484<br><br>**Prediction 3:**<br>- Text: 특정한 귀인 평향과 행동의 상관 관계를 연구<br>- Probability: 0.0152<br>- Start Logit: 8.3458<br>- End Logit: 2.8124<br><br>**Prediction 4:**<br>- Text: 평향과 행동의 상관 관계<br>- Probability: 0.0007<br>- Start Logit: 1.0932<br>- End Logit: 6.9484<br><br>**Prediction 5:**<br>- Text: 귀인 평향과 행동의 상관 관계를 연구<br>- Probability: 0.0005<br>- Start Logit: 4.9661<br>- End Logit: 2.8124 |
| mrc-0-005478 | 높은 전위의 전극 | **Prediction 1:**<br>- Text: 전극<br>- Probability: 0.5270<br>- Start Logit: 3.9255<br>- End Logit: 6.3608<br><br>**Prediction 2:**<br>- Text: 높은 전위의 전극<br>- Probability: 0.2088<br>- Start Logit: 2.9998<br>- End Logit: 6.3608<br><br>**Prediction 3:**<br>- Text: 전극으로부터 발생하는, 주로 지속적인 전류<br>- Probability: 0.0985<br>- Start Logit: 3.9255<br>- End Logit: 4.6838<br><br>**Prediction 4:**<br>- Text: 중성유체 속의 높은 전위의 전극<br>- Probability: 0.0944<br>- Start Logit: 2.2063<br>- End Logit: 6.3608<br><br>**Prediction 5:**<br>- Text: 지속적인 전류<br>- Probability: 0.0712<br>- Start Logit: 3.6002<br>- End Logit: 4.6838 |
| mrc-1-000516 | 1945년 | **Prediction 1:**<br>- Text: 1945년과 1960년대 후반<br>- Probability: 0.5606<br>- Start Logit: 7.1158<br>- End Logit: 8.0998<br><br>**Prediction 2:**<br>- Text: 1960년대 후반<br>- Probability: 0.3694<br>- Start Logit: 6.6987<br>- End Logit: 8.0998<br><br>**Prediction 3:**<br>- Text: 1945년과 1960년대 후반 사이<br>- Probability: 0.0382<br>- Start Logit: 7.1158<br>- End Logit: 5.4140<br><br>**Prediction 4:**<br>- Text: 1960년대 후반 사이<br>- Probability: 0.0252<br>- Start Logit: 6.6987<br>- End Logit: 5.4140<br><br>**Prediction 5:**<br>- Text: 1945년<br>- Probability: 0.0066<br>- Start Logit: 7.1158<br>- End Logit: 3.6542 |
| mrc-0-002679 | 브리튼인들 | **Prediction 1:**<br>- Text: 브리튼인<br>- Probability: 0.9862<br>- Start Logit: 12.3219<br>- End Logit: 10.5618<br><br>**Prediction 2:**<br>- Text: 브리튼인들<br>- Probability: 0.0101<br>- Start Logit: 12.3219<br>- End Logit: 5.9765<br><br>**Prediction 3:**<br>- Text: 브리튼<br>- Probability: 0.0035<br>- Start Logit: 12.3219<br>- End Logit: 4.9243<br><br>**Prediction 4:**<br>- Text: 브리튼인들 대한 두려움<br>- Probability: 0.0001<br>- Start Logit: 12.3219<br>- End Logit: 1.6947<br><br>**Prediction 5:**<br>- Text: 브리<br>- Probability: 0.0001<br>- Start Logit: 12.3219<br>- End Logit: 1.6821 |
| mrc-0-001980 | 1기 | **Prediction 1:**<br>- Text: 171기<br>- Probability: 0.7053<br>- Start Logit: 2.0120<br>- End Logit: 0.9322<br><br>**Prediction 2:**<br>- Text: 171<br>- Probability: 0.2314<br>- Start Logit: 2.0120<br>- End Logit: -0.1822<br><br>**Prediction 3:**<br>- Text: 모두 171기<br>- Probability: 0.0401<br>- Start Logit: -0.8561<br>- End Logit: 0.9322<br><br>**Prediction 4:**<br>- Text: 모두 171<br>- Probability: 0.0131<br>- Start Logit: -0.8561<br>- End Logit: -0.1822<br><br>**Prediction 5:**<br>- Text: 8기의 수혈유구, 1기의 돌방무덤, 1기의 띠모양유구 등으로 모두 171기<br>- Probability: 0.0100<br>- Start Logit: -2.2409<br>- End Logit: 0.9322 |
| mrc-0-004015 | 60년간 | **Prediction 1:**<br>- Text: 60년<br>- Probability: 0.5542<br>- Start Logit: 11.1113<br>- End Logit: 10.4216<br><br>**Prediction 2:**<br>- Text: 60년간<br>- Probability: 0.3466<br>- Start Logit: 11.1113<br>- End Logit: 9.9521<br><br>**Prediction 3:**<br>- Text: 4년 연상의 김옥성(金玉聲)과 결혼했다. 이후 그는 60년<br>- Probability: 0.0587<br>- Start Logit: 8.8654<br>- End Logit: 10.4216<br><br>**Prediction 4:**<br>- Text: 4년 연상의 김옥성(金玉聲)과 결혼했다. 이후 그는 60년간<br>- Probability: 0.0367<br>- Start Logit: 8.8654<br>- End Logit: 9.9521<br><br>**Prediction 5:**<br>- Text: 60<br>- Probability: 0.0038<br>- Start Logit: 11.1113<br>- End Logit: 5.4479 |
| mrc-0-000707 | 페름기 | **Prediction 1:**<br>- Text: 펜실베이니아기<br>- Probability: 0.9985<br>- Start Logit: 11.0551<br>- End Logit: 10.0600<br><br>**Prediction 2:**<br>- Text: 펜실베이니아<br>- Probability: 0.0015<br>- Start Logit: 11.0551<br>- End Logit: 3.5584<br><br>**Prediction 3:**<br>- Text: 미시시피기, 이후는 펜실베이니아기<br>- Probability: 0.0000<br>- Start Logit: 1.1498<br>- End Logit: 10.0600<br><br>**Prediction 4:**<br>- Text: 펜실베이니아기<br>- Probability: 0.0000<br>- Start Logit: 2.9722<br>- End Logit: 2.9834<br><br>**Prediction 5:**<br>- Text: 미시시피기, 이후는 펜실베이니아<br>- Probability: 0.0000<br>- Start Logit: 1.1498<br>- End Logit: 3.5584 |
| mrc-1-001446 | 대형 항공 모함 도로스 | **Prediction 1:**<br>- Text: 화이트 베이스<br>- Probability: 0.9989<br>- Start Logit: 11.3890<br>- End Logit: 10.2959<br><br>**Prediction 2:**<br>- Text: 화이트<br>- Probability: 0.0011<br>- Start Logit: 11.3890<br>- End Logit: 3.5051<br><br>**Prediction 3:**<br>- Text: 베이스<br>- Probability: 0.0000<br>- Start Logit: -0.1230<br>- End Logit: 10.2959<br><br>**Prediction 4:**<br>- Text: 화이트 베이스<br>- Probability: 0.0000<br>- Start Logit: 3.9970<br>- End Logit: 3.4531<br><br>**Prediction 5:**<br>- Text: 사라미스 급 순양함<br>- Probability: 0.0000<br>- Start Logit: 3.3135<br>- End Logit: 1.5020 |
| mrc-0-000412 | 성벽에 목이 매달리는 교수형 | **Prediction 1:**<br>- Text: 교수형<br>- Probability: 0.6196<br>- Start Logit: 11.9107<br>- End Logit: 11.6339<br><br>**Prediction 2:**<br>- Text: 교수형<br>- Probability: 0.3784<br>- Start Logit: 11.5207<br>- End Logit: 11.5306<br><br>**Prediction 3:**<br>- Text: 교수<br>- Probability: 0.0011<br>- Start Logit: 11.5207<br>- End Logit: 5.7059<br><br>**Prediction 4:**<br>- Text: 교수<br>- Probability: 0.0008<br>- Start Logit: 11.9107<br>- End Logit: 5.0416<br><br>**Prediction 5:**<br>- Text: 성벽에 목이 매달리는 교수형<br>- Probability: 0.0000<br>- Start Logit: 1.5251<br>- End Logit: 11.6339 |
| mrc-0-005355 | 본국 의회가 식민지에 대한 입법을 “어떤 경우에도 행할 수” 있도록 하였다 | **Prediction 1:**<br>- Text: 주권<br>- Probability: 0.9822<br>- Start Logit: 6.6258<br>- End Logit: 6.2805<br><br>**Prediction 2:**<br>- Text: 본국 의회가 식민지에 대한 주권<br>- Probability: 0.0081<br>- Start Logit: 1.8311<br>- End Logit: 6.2805<br><br>**Prediction 3:**<br>- Text: 주권을 행사<br>- Probability: 0.0069<br>- Start Logit: 6.6258<br>- End Logit: 1.3204<br><br>**Prediction 4:**<br>- Text: 식민지에 대한 주권<br>- Probability: 0.0025<br>- Start Logit: 0.6531<br>- End Logit: 6.2805<br><br>**Prediction 5:**<br>- Text: 입법<br>- Probability: 0.0003<br>- Start Logit: 2.6641<br>- End Logit: 2.0758 |
| mrc-1-000899 | 보은군 남쪽 상용리(현 영동군 용산면 상용리) 마을 | **Prediction 1:**<br>- Text: 상용리(현 영동군 용산면 상용리) 마을 근처<br>- Probability: 0.7457<br>- Start Logit: 12.2291<br>- End Logit: 9.7003<br><br>**Prediction 2:**<br>- Text: 상용리(현 영동군 용산면 상용리) 마을<br>- Probability: 0.2349<br>- Start Logit: 12.2291<br>- End Logit: 8.5450<br><br>**Prediction 3:**<br>- Text: 황간<br>- Probability: 0.0113<br>- Start Logit: 9.3123<br>- End Logit: 8.4260<br><br>**Prediction 4:**<br>- Text: 상용리<br>- Probability: 0.0070<br>- Start Logit: 12.2291<br>- End Logit: 5.0315<br><br>**Prediction 5:**<br>- Text: 보은군 남쪽 상용리(현 영동군 용산면 상용리) 마을 근처<br>- Probability: 0.0012<br>- Start Logit: 5.7753<br>- End Logit: 9.7003 |
| mrc-0-002886 | 태화관(서울시 종로구 인사동 소재) | **Prediction 1:**<br>- Text: 태화관<br>- Probability: 0.8733<br>- Start Logit: 12.1860<br>- End Logit: 11.4384<br><br>**Prediction 2:**<br>- Text: 태화관(서울시 종로구 인사동 소재)<br>- Probability: 0.1260<br>- Start Logit: 12.1860<br>- End Logit: 9.5024<br><br>**Prediction 3:**<br>- Text: 태화<br>- Probability: 0.0007<br>- Start Logit: 12.1860<br>- End Logit: 4.2943<br><br>**Prediction 4:**<br>- Text: 관<br>- Probability: 0.0000<br>- Start Logit: -0.0461<br>- End Logit: 11.4384<br><br>**Prediction 5:**<br>- Text: 관(서울시 종로구 인사동 소재)<br>- Probability: 0.0000<br>- Start Logit: -0.0461<br>- End Logit: 9.5024 |
| mrc-0-004454 | 식재료 | **Prediction 1:**<br>- Text: 같은 회사의 현상금 사냥꾼들의 식재료를 구하기 위해 거대 고래<br>- Probability: 0.2451<br>- Start Logit: 8.1472<br>- End Logit: 7.7580<br><br>**Prediction 2:**<br>- Text: 같은 회사의 현상금 사냥꾼들의 식재료<br>- Probability: 0.2214<br>- Start Logit: 8.1472<br>- End Logit: 7.6564<br><br>**Prediction 3:**<br>- Text: 현상금 사냥꾼들의 식재료를 구하기 위해 거대 고래<br>- Probability: 0.2012<br>- Start Logit: 7.9497<br>- End Logit: 7.7580<br><br>**Prediction 4:**<br>- Text: 현상금 사냥꾼들의 식재료<br>- Probability: 0.1818<br>- Start Logit: 7.9497<br>- End Logit: 7.6564<br><br>**Prediction 5:**<br>- Text: 거대 고래<br>- Probability: 0.1505<br>- Start Logit: 7.6593<br>- End Logit: 7.7580 |
| mrc-1-001534 | 공정 | **Prediction 1:**<br>- Text: 거북이가 토끼를 보고도 그냥 뛰어갔기에 공정하지 않으므로<br>- Probability: 0.5520<br>- Start Logit: 9.2815<br>- End Logit: 3.4269<br><br>**Prediction 2:**<br>- Text: 거북이가 토끼를 보고도 그냥 뛰어갔기에 공정<br>- Probability: 0.4270<br>- Start Logit: 9.2815<br>- End Logit: 3.1700<br><br>**Prediction 3:**<br>- Text: 거북이<br>- Probability: 0.0160<br>- Start Logit: 9.2815<br>- End Logit: -0.1136<br><br>**Prediction 4:**<br>- Text: 공정하지 않으므로<br>- Probability: 0.0028<br>- Start Logit: 4.0064<br>- End Logit: 3.4269<br><br>**Prediction 5:**<br>- Text: 공정<br>- Probability: 0.0022<br>- Start Logit: 4.0064<br>- End Logit: 3.1700 |
| mrc-1-000918 | 좌우대칭 | **Prediction 1:**<br>- Text: 좌우대칭의 몸<br>- Probability: 0.5259<br>- Start Logit: 11.8968<br>- End Logit: 8.0179<br><br>**Prediction 2:**<br>- Text: 좌우대칭<br>- Probability: 0.4739<br>- Start Logit: 11.8968<br>- End Logit: 7.9138<br><br>**Prediction 3:**<br>- Text: 좌우<br>- Probability: 0.0002<br>- Start Logit: 11.8968<br>- End Logit: 0.2909<br><br>**Prediction 4:**<br>- Text: 좌우대칭의 몸을 가지고 있었던 것<br>- Probability: 0.0000<br>- Start Logit: 11.8968<br>- End Logit: -2.5748<br><br>**Prediction 5:**<br>- Text: 좌우대칭<br>- Probability: 0.0000<br>- Start Logit: 4.5796<br>- End Logit: 3.0323 |
| mrc-1-001611 | 바라나시 | **Prediction 1:**<br>- Text: 라자그리하<br>- Probability: 0.9999<br>- Start Logit: 12.4628<br>- End Logit: 12.0776<br><br>**Prediction 2:**<br>- Text: 라자그리<br>- Probability: 0.0001<br>- Start Logit: 12.4628<br>- End Logit: 2.6902<br><br>**Prediction 3:**<br>- Text: 라자<br>- Probability: 0.0000<br>- Start Logit: 12.4628<br>- End Logit: 2.0494<br><br>**Prediction 4:**<br>- Text: 하<br>- Probability: 0.0000<br>- Start Logit: 0.8733<br>- End Logit: 12.0776<br><br>**Prediction 5:**<br>- Text: 기리브라자 인근에 있는 철광산 지역에 약 40킬로미터나 되는 성벽으로 에워싸인 신도시인 라자그리하<br>- Probability: 0.0000<br>- Start Logit: 0.4546<br>- End Logit: 12.0776 |
| mrc-1-001285 | 우리의 훌륭한 중국 동료 | **Prediction 1:**<br>- Text: 캠벨p=119<br>- Probability: 0.7815<br>- Start Logit: 5.2163<br>- End Logit: 4.7848<br><br>**Prediction 2:**<br>- Text: 중국 동료<br>- Probability: 0.1300<br>- Start Logit: 3.9645<br>- End Logit: 4.2426<br><br>**Prediction 3:**<br>- Text: 훌륭한 중국 동료<br>- Probability: 0.0502<br>- Start Logit: 3.0125<br>- End Logit: 4.2426<br><br>**Prediction 4:**<br>- Text: 우리의 훌륭한 중국 동료<br>- Probability: 0.0348<br>- Start Logit: 2.6473<br>- End Logit: 4.2426<br><br>**Prediction 5:**<br>- Text: 중국 동료는 형제들로부터 진심어린 환영<br>- Probability: 0.0035<br>- Start Logit: 3.9645<br>- End Logit: 0.6360 |
| mrc-0-003587 | 사춘기에서 흔히 볼 수 있는 정서적 불안정성 | **Prediction 1:**<br>- Text: 불안감<br>- Probability: 0.7239<br>- Start Logit: -2.2361<br>- End Logit: -1.7503<br><br>**Prediction 2:**<br>- Text: 미세한 변화<br>- Probability: 0.1025<br>- Start Logit: -2.3685<br>- End Logit: -3.5730<br><br>**Prediction 3:**<br>- Text: 조현병<br>- Probability: 0.0667<br>- Start Logit: -3.4127<br>- End Logit: -2.9575<br><br>**Prediction 4:**<br>- Text: 미세한 변화를 눈치 채기가 어렵다. 가장 기본적인 변화는 일상생활의 여러 영역에서 일탈현상<br>- Probability: 0.0644<br>- Start Logit: -2.3685<br>- End Logit: -4.0367<br><br>**Prediction 5:**<br>- Text: 눈치 채기가 어렵다. 가장 기본적인 변화는 일상생활의 여러 영역에서 일탈현상<br>- Probability: 0.0425<br>- Start Logit: -2.7854<br>- End Logit: -4.0367 |
| mrc-0-003022 | 아일랜드 선교자 골룸바노 | **Prediction 1:**<br>- Text: 골룸바노<br>- Probability: 0.7511<br>- Start Logit: 12.3383<br>- End Logit: 12.0101<br><br>**Prediction 2:**<br>- Text: 골룸바노<br>- Probability: 0.1823<br>- Start Logit: 11.3193<br>- End Logit: 11.6134<br><br>**Prediction 3:**<br>- Text: 아일랜드 선교자 골룸바노<br>- Probability: 0.0534<br>- Start Logit: 10.0908<br>- End Logit: 11.6134<br><br>**Prediction 4:**<br>- Text: 아일랜드 선교자 골룸바노<br>- Probability: 0.0132<br>- Start Logit: 8.2976<br>- End Logit: 12.0101<br><br>**Prediction 5:**<br>- Text: 골룸바<br>- Probability: 0.0000<br>- Start Logit: 12.3383<br>- End Logit: 2.3332 |
| mrc-0-003211 | 할머니 | **Prediction 1:**<br>- Text: 안드레우치오 디 피에트로<br>- Probability: 0.8334<br>- Start Logit: 7.5039<br>- End Logit: 6.7806<br><br>**Prediction 2:**<br>- Text: 안드레우치오 디 피에트로를 지켜보았는데, 그가 우연히 예전에 알던 한 할머니<br>- Probability: 0.1658<br>- Start Logit: 7.5039<br>- End Logit: 5.1657<br><br>**Prediction 3:**<br>- Text: 할머니<br>- Probability: 0.0007<br>- Start Logit: 2.0819<br>- End Logit: 5.1657<br><br>**Prediction 4:**<br>- Text: 안드레우치오 디 피에트로<br>- Probability: 0.0001<br>- Start Logit: 1.9790<br>- End Logit: 3.1100<br><br>**Prediction 5:**<br>- Text: 할머니를 만나 반가워하는 것을 보고, 나중에 그 할머니<br>- Probability: 0.0000<br>- Start Logit: 2.0819<br>- End Logit: 2.0782 |
| mrc-1-001173 | 탄광 | **Prediction 1:**<br>- Text: 탄광 환경 문제<br>- Probability: 0.8677<br>- Start Logit: 12.4856<br>- End Logit: 11.6221<br><br>**Prediction 2:**<br>- Text: 탄광 환경 문제<br>- Probability: 0.1295<br>- Start Logit: 11.4117<br>- End Logit: 10.7936<br><br>**Prediction 3:**<br>- Text: 인도 보팔시의 가스 누출 참사를 다룬\n장시 <눈먼 자를 위하여>, 탄광 환경 문제<br>- Probability: 0.0028<br>- Start Logit: 7.5743<br>- End Logit: 10.7936<br><br>**Prediction 4:**<br>- Text: 탄광<br>- Probability: 0.0000<br>- Start Logit: 12.4856<br>- End Logit: 1.0629<br><br>**Prediction 5:**<br>- Text: 탄광 환경<br>- Probability: 0.0000<br>- Start Logit: 12.4856<br>- End Logit: 0.7313 |
| mrc-1-000593 | 마거릿 대처 전 영국수상 | **Prediction 1:**<br>- Text: 마거릿 대처<br>- Probability: 0.7279<br>- Start Logit: 12.6226<br>- End Logit: 10.8081<br><br>**Prediction 2:**<br>- Text: 마거릿 대처 전 영국수상<br>- Probability: 0.2720<br>- Start Logit: 12.6226<br>- End Logit: 9.8238<br><br>**Prediction 3:**<br>- Text: 마<br>- Probability: 0.0000<br>- Start Logit: 12.6226<br>- End Logit: 1.1279<br><br>**Prediction 4:**<br>- Text: 마거릿<br>- Probability: 0.0000<br>- Start Logit: 12.6226<br>- End Logit: -0.3624<br><br>**Prediction 5:**<br>- Text: 대처<br>- Probability: 0.0000<br>- Start Logit: 1.1953<br>- End Logit: 10.8081 |
| mrc-1-000132 | 트르피미로비치 왕조(Trpimirović) 출신의 문치미르 | **Prediction 1:**<br>- Text: 문치미르<br>- Probability: 0.9998<br>- Start Logit: 11.3558<br>- End Logit: 11.0527<br><br>**Prediction 2:**<br>- Text: 트르피미로비치 왕조(Trpimirović) 출신의 문치미르<br>- Probability: 0.0002<br>- Start Logit: 2.5967<br>- End Logit: 11.0527<br><br>**Prediction 3:**<br>- Text: 문치미<br>- Probability: 0.0001<br>- Start Logit: 11.3558<br>- End Logit: 1.1740<br><br>**Prediction 4:**<br>- Text: 르<br>- Probability: 0.0000<br>- Start Logit: 0.7405<br>- End Logit: 11.0527<br><br>**Prediction 5:**<br>- Text: 문<br>- Probability: 0.0000<br>- Start Logit: 11.3558<br>- End Logit: -0.6046 |
| mrc-1-001121 | 군대 | **Prediction 1:**<br>- Text: 가능한 다양한 미래 시나리오의 도전을 해결할 수있는 군대<br>- Probability: 0.8931<br>- Start Logit: 9.1878<br>- End Logit: 9.4145<br><br>**Prediction 2:**<br>- Text: 다양한 미래 시나리오의 도전을 해결할 수있는 군대<br>- Probability: 0.1047<br>- Start Logit: 7.0440<br>- End Logit: 9.4145<br><br>**Prediction 3:**<br>- Text: 미래 시나리오의 도전을 해결할 수있는 군대<br>- Probability: 0.0019<br>- Start Logit: 3.0420<br>- End Logit: 9.4145<br><br>**Prediction 4:**<br>- Text: 군대<br>- Probability: 0.0003<br>- Start Logit: 1.0831<br>- End Logit: 9.4145<br><br>**Prediction 5:**<br>- Text: 가능한 다양한 미래 시나리오의 도전을 해결할 수있는 군대를 기반으로 한다.<br>- Probability: 0.0000<br>- Start Logit: 9.1878<br>- End Logit: -2.1116 |
| mrc-0-004725 | 1966년 | **Prediction 1:**<br>- Text: 1956년<br>- Probability: 0.9525<br>- Start Logit: 9.3179<br>- End Logit: 9.6146<br><br>**Prediction 2:**<br>- Text: 1956년<br>- Probability: 0.0333<br>- Start Logit: 7.4969<br>- End Logit: 8.0815<br><br>**Prediction 3:**<br>- Text: 1956<br>- Probability: 0.0135<br>- Start Logit: 9.3179<br>- End Logit: 5.3603<br><br>**Prediction 4:**<br>- Text: 1956<br>- Probability: 0.0006<br>- Start Logit: 7.4969<br>- End Logit: 4.0347<br><br>**Prediction 5:**<br>- Text: 년<br>- Probability: 0.0001<br>- Start Logit: -0.1475<br>- End Logit: 9.6146 |
| mrc-1-000507 | 17세기 조선 시대 | **Prediction 1:**<br>- Text: 17세기<br>- Probability: 0.6387<br>- Start Logit: 11.3741<br>- End Logit: 9.0498<br><br>**Prediction 2:**<br>- Text: 17세기 조선 시대<br>- Probability: 0.1915<br>- Start Logit: 10.8209<br>- End Logit: 8.3983<br><br>**Prediction 3:**<br>- Text: 17세기 조선<br>- Probability: 0.1071<br>- Start Logit: 11.3741<br>- End Logit: 7.2640<br><br>**Prediction 4:**<br>- Text: 17세기 조선 시대<br>- Probability: 0.0351<br>- Start Logit: 11.3741<br>- End Logit: 6.1489<br><br>**Prediction 5:**<br>- Text: 17세기<br>- Probability: 0.0277<br>- Start Logit: 10.8209<br>- End Logit: 6.4644 |
| mrc-0-001719 | 모 | **Prediction 1:**<br>- Text: 창<br>- Probability: 0.9794<br>- Start Logit: -1.0412<br>- End Logit: -0.5207<br><br>**Prediction 2:**<br>- Text: 극<br>- Probability: 0.0076<br>- Start Logit: -3.1097<br>- End Logit: -3.3115<br><br>**Prediction 3:**<br>- Text: 과와 같이 걸어 자르거나 또는 모<br>- Probability: 0.0050<br>- Start Logit: -3.3674<br>- End Logit: -3.4732<br><br>**Prediction 4:**<br>- Text: 극은 때에 따라 과와 같이 걸어 자르거나 또는 모<br>- Probability: 0.0044<br>- Start Logit: -3.4944<br>- End Logit: -3.4732<br><br>**Prediction 5:**<br>- Text: 극<br>- Probability: 0.0036<br>- Start Logit: -3.4944<br>- End Logit: -3.6803 |
| mrc-0-001552 | 비단 | **Prediction 1:**<br>- Text: 석지 채용신<br>- Probability: 0.9837<br>- Start Logit: 9.2195<br>- End Logit: 9.3896<br><br>**Prediction 2:**<br>- Text: 석지 채용신이 그린 작품으로, 비단<br>- Probability: 0.0115<br>- Start Logit: 9.2195<br>- End Logit: 4.9378<br><br>**Prediction 3:**<br>- Text: 채용신<br>- Probability: 0.0045<br>- Start Logit: 3.8291<br>- End Logit: 9.3896<br><br>**Prediction 4:**<br>- Text: 비단<br>- Probability: 0.0003<br>- Start Logit: 5.6678<br>- End Logit: 4.9378<br><br>**Prediction 5:**<br>- Text: 석지<br>- Probability: 0.0001<br>- Start Logit: 9.2195<br>- End Logit: -0.3230 |
| mrc-0-004530 | 모든 관직을 거친 대과 급제자 | **Prediction 1:**<br>- Text: 대과 급제자<br>- Probability: 0.5440<br>- Start Logit: 2.9608<br>- End Logit: 5.6408<br><br>**Prediction 2:**<br>- Text: 학문과 관련되는 모든 관직을 거친 대과 급제자<br>- Probability: 0.4328<br>- Start Logit: 2.7321<br>- End Logit: 5.6408<br><br>**Prediction 3:**<br>- Text: 대과급제자<br>- Probability: 0.0098<br>- Start Logit: 1.7135<br>- End Logit: 2.8712<br><br>**Prediction 4:**<br>- Text: 대과 급제자만<br>- Probability: 0.0075<br>- Start Logit: 2.9608<br>- End Logit: 1.3545<br><br>**Prediction 5:**<br>- Text: 학문과 관련되는 모든 관직을 거친 대과 급제자만<br>- Probability: 0.0060<br>- Start Logit: 2.7321<br>- End Logit: 1.3545 |
| mrc-0-003850 | 제1부인 | **Prediction 1:**<br>- Text: 태조 왕건<br>- Probability: 0.9937<br>- Start Logit: 4.1246<br>- End Logit: 4.0616<br><br>**Prediction 2:**<br>- Text: 왕건<br>- Probability: 0.0035<br>- Start Logit: -1.5152<br>- End Logit: 4.0616<br><br>**Prediction 3:**<br>- Text: 태조<br>- Probability: 0.0017<br>- Start Logit: 4.1246<br>- End Logit: -2.3374<br><br>**Prediction 4:**<br>- Text: 복지겸과 함께 몰래 모의하여 밤에 태조 왕건<br>- Probability: 0.0009<br>- Start Logit: -2.9098<br>- End Logit: 4.0616<br><br>**Prediction 5:**<br>- Text: 태조 왕건의 집<br>- Probability: 0.0003<br>- Start Logit: 4.1246<br>- End Logit: -4.1859 |
| mrc-0-004662 | 지질 과산화 생성물 | **Prediction 1:**<br>- Text: 지질<br>- Probability: 0.9909<br>- Start Logit: 9.9545<br>- End Logit: 8.6687<br><br>**Prediction 2:**<br>- Text: 지질 과산화 생성물<br>- Probability: 0.0089<br>- Start Logit: 9.9545<br>- End Logit: 3.9567<br><br>**Prediction 3:**<br>- Text: 지질 과산화 생성물이 혼합<br>- Probability: 0.0002<br>- Start Logit: 9.9545<br>- End Logit: 0.1868<br><br>**Prediction 4:**<br>- Text: 지질 과산화<br>- Probability: 0.0000<br>- Start Logit: 9.9545<br>- End Logit: -1.9885<br><br>**Prediction 5:**<br>- Text: 아라키돈산의 섭취 증가는 지질<br>- Probability: 0.0000<br>- Start Logit: -1.7409<br>- End Logit: 8.6687 |
| mrc-0-001162 | 퀘이커 교도였던 앨리스 페어살 스미스 | **Prediction 1:**<br>- Text: 앨리스 페어살 스미스<br>- Probability: 0.9956<br>- Start Logit: 11.5776<br>- End Logit: 10.2742<br><br>**Prediction 2:**<br>- Text: 존 러셀 백작의 손자<br>- Probability: 0.0037<br>- Start Logit: 7.9421<br>- End Logit: 8.3030<br><br>**Prediction 3:**<br>- Text: 퀘이커 교도였던 앨리스 페어살 스미스<br>- Probability: 0.0004<br>- Start Logit: 3.6934<br>- End Logit: 10.2742<br><br>**Prediction 4:**<br>- Text: 러셀은 17세에 처음으로 퀘이커 교도였던 앨리스 페어살 스미스<br>- Probability: 0.0002<br>- Start Logit: 3.0053<br>- End Logit: 10.2742<br><br>**Prediction 5:**<br>- Text: 앨리스<br>- Probability: 0.0002<br>- Start Logit: 11.5776<br>- End Logit: 1.5475 |
| mrc-0-004565 | 다수결 | **Prediction 1:**<br>- Text: 주어진 방법과 같은 방법<br>- Probability: 0.4885<br>- Start Logit: 2.1162<br>- End Logit: 3.0684<br><br>**Prediction 2:**<br>- Text: 주어진 방법<br>- Probability: 0.2094<br>- Start Logit: 2.1162<br>- End Logit: 2.2212<br><br>**Prediction 3:**<br>- Text: 대다수의 부족이 주어진 방법과 같은 방법<br>- Probability: 0.1485<br>- Start Logit: 0.9255<br>- End Logit: 3.0684<br><br>**Prediction 4:**<br>- Text: 같은 방법<br>- Probability: 0.0899<br>- Start Logit: 0.4241<br>- End Logit: 3.0684<br><br>**Prediction 5:**<br>- Text: 대다수의 부족이 주어진 방법<br>- Probability: 0.0637<br>- Start Logit: 0.9255<br>- End Logit: 2.2212 |
| mrc-0-003083 | 미나미 지로 | **Prediction 1:**<br>- Text: 미나미 지로 총독<br>- Probability: 0.9873<br>- Start Logit: 11.1136<br>- End Logit: 9.8159<br><br>**Prediction 2:**<br>- Text: 미나미 지로 총독<br>- Probability: 0.0078<br>- Start Logit: 8.1649<br>- End Logit: 7.9178<br><br>**Prediction 3:**<br>- Text: 윤치호의 청을 받아들인 미나미 지로 총독<br>- Probability: 0.0027<br>- Start Logit: 5.2150<br>- End Logit: 9.8159<br><br>**Prediction 4:**<br>- Text: 미나미 지로<br>- Probability: 0.0021<br>- Start Logit: 11.1136<br>- End Logit: 3.6576<br><br>**Prediction 5:**<br>- Text: 미나미 지로<br>- Probability: 0.0001<br>- Start Logit: 8.1649<br>- End Logit: 3.5426 |
| mrc-1-000622 | 공산주의 | **Prediction 1:**<br>- Text: 오귀스트 블랑키(Auguste Blanqui)의 비밀결사주의<br>- Probability: 0.9936<br>- Start Logit: 0.4604<br>- End Logit: 2.9488<br><br>**Prediction 2:**<br>- Text: 블랑키주의<br>- Probability: 0.0031<br>- Start Logit: -1.7329<br>- End Logit: -0.6142<br><br>**Prediction 3:**<br>- Text: 블랑키주의<br>- Probability: 0.0021<br>- Start Logit: -2.1076<br>- End Logit: -0.6635<br><br>**Prediction 4:**<br>- Text: 블랑키주의<br>- Probability: 0.0010<br>- Start Logit: -2.4366<br>- End Logit: -1.0765<br><br>**Prediction 5:**<br>- Text: 블랑키주의<br>- Probability: 0.0003<br>- Start Logit: -1.9774<br>- End Logit: -2.8404 |


In [17]:
import json
from datasets import load_dataset
import scipy.special
import pandas as pd

# 데이터셋 경로와 n-best 예측값 파일 경로 설정
dataset_path = "/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/data/default/validation/data-00000-of-00001.arrow"
nbest_pred_path = "/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/outputs/original_default_bz=16_lr=1.6764783497920226e-05_eval_nbest_predictions.json"

# 스코어링 함수에 사용할 γ 값 설정
gamma = 0.3  # 필요에 따라 조정

# 데이터셋 로드
dataset = load_dataset('arrow', data_files=dataset_path, split='train')

# n-best 예측값 로드
with open(nbest_pred_path, 'r', encoding='utf-8') as f:
    nbest_preds = json.load(f)

# 정확도 계산을 위한 변수 초기화
total_examples = 0
correct_original = 0
correct_new = 0

# 틀린 예측을 저장할 리스트 초기화
wrong_cases = []

# 데이터셋의 각 예제에 대해 반복
for example in dataset:
    total_examples += 1
    example_id = example['id']
    # 정답은 리스트 형태로 저장되어 있다고 가정
    correct_answers = example['answers']['text']
    
    # 해당 예제의 n-best 예측값 가져오기
    preds = nbest_preds.get(example_id, [])
    if not preds:
        continue  # 예측값이 없는 경우 건너뜀
    
    # 기존 스코어링 방식의 최상위 예측값
    top_pred_original = preds[0]['text'].strip().lower()
    
    # 정답과 비교 (대소문자 무시)
    is_correct_original = any(top_pred_original == ans.strip().lower() for ans in correct_answers)
    if is_correct_original:
        correct_original += 1
    
    # 새로운 스코어링 방식 (Log-Softmax 적용)
    # 시작 로짓과 종료 로짓을 수집
    start_logits = [pred.get('start_logit', 0.0) for pred in preds]
    end_logits = [pred.get('end_logit', 0.0) for pred in preds]
    
    # Log-Softmax 적용
    log_prob_start = scipy.special.log_softmax(start_logits)
    log_prob_end = scipy.special.log_softmax(end_logits)
    
    # 각 예측에 스코어 계산 및 저장
    for idx, pred in enumerate(preds):
        pred['log_prob_start'] = log_prob_start[idx]
        pred['log_prob_end'] = log_prob_end[idx]
        pred['score'] = (
            pred['log_prob_start'] +
            pred['log_prob_end'] +
            gamma * (pred['log_prob_end'] - pred['log_prob_start'])
        )
    
    # 새로운 스코어링 방식으로 정렬 (내림차순)
    sorted_preds_new = sorted(preds, key=lambda x: x['score'], reverse=True)
    
    # 새로운 스코어링 방식의 최상위 예측값
    top_pred_new = sorted_preds_new[0]['text'].strip().lower()
    
    # 정답과 비교 (대소문자 무시)
    is_correct_new = any(top_pred_new == ans.strip().lower() for ans in correct_answers)
    if is_correct_new:
        correct_new += 1
    
    # 기존 방식에서 틀린 예측인 경우에 대해 n-best 예측값을 수집
    if not is_correct_original:
        # 원래 n-best 예측값 포맷팅
        nbest_original_details = []
        for idx, pred in enumerate(preds, 1):
            detail = (
                f"**Prediction {idx}:**<br>"
                f"- Text: {pred['text']}<br>"
                f"- Probability: {pred['probability']:.4f}<br>"
                f"- Start Logit: {pred['start_logit']:.4f}<br>"
                f"- End Logit: {pred['end_logit']:.4f}"
            )
            nbest_original_details.append(detail)
        nbest_original_combined = "<br><br>".join(nbest_original_details)
        
        # 새로운 스코어링 방식으로 정렬된 n-best 예측값 포맷팅
        nbest_new_details = []
        for idx, pred in enumerate(sorted_preds_new, 1):
            detail = (
                f"**Prediction {idx}:**<br>"
                f"- Text: {pred['text']}<br>"
                f"- Probability: {pred['probability']:.4f}<br>"
                f"- Start Logit: {pred['start_logit']:.4f}<br>"
                f"- End Logit: {pred['end_logit']:.4f}<br>"
                f"- Score: {pred['score']:.4f}"
            )
            nbest_new_details.append(detail)
        nbest_new_combined = "<br><br>".join(nbest_new_details)
        
        # 틀린 사례 저장
        wrong_cases.append({
            'ID': example_id,
            '정답': "; ".join(correct_answers),
            'Original N-best 예측값': nbest_original_combined,
            'New Scoring N-best 예측값': nbest_new_combined
        })

# 정확도 계산
accuracy_original = (correct_original / total_examples) * 100 if total_examples > 0 else 0
accuracy_new = (correct_new / total_examples) * 100 if total_examples > 0 else 0

# 정확도 비교 표 생성
accuracy_comparison = {
    'Metric': ['정확도 (%)'],
    '기존 스코어링': [f"{accuracy_original:.2f}"],
    '새로운 스코어링 (Log-Softmax)': [f"{accuracy_new:.2f}"]
}

# 틀린 예측에 대한 상세 정보가 있는지 확인
if wrong_cases:
    df_wrong = pd.DataFrame(wrong_cases)
else:
    df_wrong = pd.DataFrame(columns=['ID', '정답', 'Original N-best 예측값', 'New Scoring N-best 예측값'])

# 마크다운 표 생성 함수
def create_markdown_table(headers, rows):
    header_line = "| " + " | ".join(headers) + " |"
    separator_line = "| " + " | ".join(["---"] * len(headers)) + " |"
    row_lines = [f"| {row} |" for row in rows]
    return "\n".join([header_line, separator_line] + row_lines)

# 정확도 비교 마크다운 표 생성
headers_accuracy = ["Metric", "기존 스코어링", "새로운 스코어링 (Log-Softmax)"]
rows_accuracy = [f"{accuracy_comparison['Metric'][0]} | {accuracy_comparison['기존 스코어링'][0]} | {accuracy_comparison['새로운 스코어링 (Log-Softmax)'][0]}"]
markdown_table_accuracy = create_markdown_table(headers_accuracy, rows_accuracy)

# 틀린 예측에 대한 상세 마크다운 표 생성
if not df_wrong.empty:
    headers_wrong = ["ID", "정답", "Original N-best 예측값", "New Scoring N-best 예측값"]
    rows_wrong = []
    for _, row in df_wrong.iterrows():
        row_content = f"{row['ID']} | {row['정답']} | {row['Original N-best 예측값']} | {row['New Scoring N-best 예측값']} |"
        rows_wrong.append(row_content)
    markdown_table_wrong = create_markdown_table(headers_wrong, rows_wrong)
else:
    markdown_table_wrong = "모델이 틀린 예측을 하지 않았습니다."

# 최종 마크다운 생성
final_markdown = f"""
## 예측 정확도 비교

{markdown_table_accuracy}

## 틀린 예측에 대한 N-best 예측값 비교

{markdown_table_wrong}
"""

print(final_markdown)

# # (선택 사항) 마크다운 표를 파일로 저장
# with open("prediction_comparison_with_log_softmax.md", "w", encoding="utf-8") as md_file:
#     md_file.write(final_markdown)

# print("마크다운 표가 'prediction_comparison_with_log_softmax.md' 파일로 저장되었습니다.")



## 예측 정확도 비교

| Metric | 기존 스코어링 | 새로운 스코어링 (Log-Softmax) |
| --- | --- | --- |
| 정확도 (%) | 71.67 | 71.67 |

## 틀린 예측에 대한 N-best 예측값 비교

| ID | 정답 | Original N-best 예측값 | New Scoring N-best 예측값 |
| --- | --- | --- | --- |
| mrc-0-005106 | 옥음방송 | **Prediction 1:**<br>- Text: 쇼와 천황의 옥음방송<br>- Probability: 0.9660<br>- Start Logit: 8.1817<br>- End Logit: 7.8816<br><br>**Prediction 2:**<br>- Text: 옥음방송<br>- Probability: 0.0328<br>- Start Logit: 4.7979<br>- End Logit: 7.8816<br><br>**Prediction 3:**<br>- Text: 쇼와 천황의 옥음방송(라디오 방송)<br>- Probability: 0.0011<br>- Start Logit: 8.1817<br>- End Logit: 1.1315<br><br>**Prediction 4:**<br>- Text: 천황의 옥음방송<br>- Probability: 0.0000<br>- Start Logit: -1.8940<br>- End Logit: 7.8816<br><br>**Prediction 5:**<br>- Text: 옥음방송(라디오 방송)<br>- Probability: 0.0000<br>- Start Logit: 4.7979<br>- End Logit: 1.1315 | **Prediction 1:**<br>- Text: 쇼와 천황의 옥음방송<br>- Probability: 0.9660<br>- Start Logit: 8.1817<br>- End Logit: 7.8816<br>- Score: -1.9378<br><br>**Prediction